In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

339


In [3]:
df = pd.read_csv("../dataset/본문_형태소분석/2019산림복지_본문_형태소분석(형태소만).csv", encoding='UTF-8')
df

,split_str,org_idx,correct_str,morphs
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,"['코리아', '투데이', '뉴스', '상주시', '상주시', '2020년', '상..."
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...,"['이소영', '남혜진', '박소영', '상하', '수도사', '업소', '김인',..."
2,안녕하세요 메리썽입니다,1,안녕하세요. 메리 성입니다,"['안녕하세요', '.', '메리', '성', '입니다']"
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요,"['혁신도시', '에', '대해', '서는', '많이', '들', '들어', '보셨..."
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...,"['현재', '혁신도시', '시즌', '2', '가', '지속', '해', '진행'..."
...,...,...,...,...
164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...,"['의', '건립', '비', '로', '지출', '하는', '기', '부금', '..."
164723,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...,"['이하', '이', '호', '에서', '같다가', '추천', '하는', '자',..."
164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,"['21', '근로자', '직업', '능력', '개발', '법', '시행', '령'..."
164725,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...,"['한국', '국제', '학교', '22', '천진', '한국', '국제', '학교..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,correct_str,morphs,joined_morphs
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,"['코리아', '투데이', '뉴스', '상주시', '상주시', '2020년', '상...",코리아 투데이 뉴스 상주시 상주시 2020년 상반기 정기 인사 이동 조서 2020 ...
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...,"['이소영', '남혜진', '박소영', '상하', '수도사', '업소', '김인',...",이소영 남혜진 박소영 상하 수도사 업소 김인 화 서 원상 김주 헌 윤 원해 평생 학...
2,안녕하세요 메리썽입니다,1,안녕하세요. 메리 성입니다,"['안녕하세요', '.', '메리', '성', '입니다']",안녕하세요 . 메리 성 입니다
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요,"['혁신도시', '에', '대해', '서는', '많이', '들', '들어', '보셨...",혁신도시 에 대해 서는 많이 들 들어 보셨을 거 라고 생각 이 드는데요
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...,"['현재', '혁신도시', '시즌', '2', '가', '지속', '해', '진행'...",현재 혁신도시 시즌 2 가 지속 해 진행 되고 있는 만큼 지금 시기 에 이 에 대해...
...,...,...,...,...,...
164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...,"['의', '건립', '비', '로', '지출', '하는', '기', '부금', '...",의 건립 비 로 지출 하는 기 부금 9 중소기업 협동조합 법 에 따른 중소기업 중앙...
164723,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...,"['이하', '이', '호', '에서', '같다가', '추천', '하는', '자',...",이하 이 호 에서 같다가 추천 하는 자 에게 지출 하 거나 대 한 체육회 에 운동선...
164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,"['21', '근로자', '직업', '능력', '개발', '법', '시행', '령'...",21 근로자 직업 능력 개발 법 시행 령 제 2조 에 따른 공공 단체 에 근로자 훈...
164725,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...,"['한국', '국제', '학교', '22', '천진', '한국', '국제', '학교...",한국 국제 학교 22 천진 한국 국제 학교 23 테헤란 한국 학교 ㅇ 법정 기 부금...


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,correct_str,morphs,joined_morphs,joined_sentences
0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,0,코리아투데이뉴스 상주시 상주시 2020년 상반기 정기인사 이동조서 2020 1 1 ...,"['코리아', '투데이', '뉴스', '상주시', '상주시', '2020년', '상...",코리아 투데이 뉴스 상주시 상주시 2020년 상반기 정기 인사 이동 조서 2020 ...,코리아 투데이 뉴스 상주시 상주시 2020년 상반기 정기 인사 이동 조서 2020 ...
1,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤원해 평생학습원 이세인 ...,0,이소영 남혜진 박소영 상하수도사업소 김인화 서원상 김주헌 윤 원해 평생학습원 이세인...,"['이소영', '남혜진', '박소영', '상하', '수도사', '업소', '김인',...",이소영 남혜진 박소영 상하 수도사 업소 김인 화 서 원상 김주 헌 윤 원해 평생 학...,코리아 투데이 뉴스 상주시 상주시 2020년 상반기 정기 인사 이동 조서 2020 ...
2,안녕하세요 메리썽입니다,1,안녕하세요. 메리 성입니다,"['안녕하세요', '.', '메리', '성', '입니다']",안녕하세요 . 메리 성 입니다,안녕하세요 . 메리 성 입니다 혁신도시 에 대해 서는 많이 들 들어 보셨을 거 라고...
3,혁신도시에 대해서는 많이들 들어보셨을거라고 생각이 드는데요,1,혁신도시에 대해서는 많이들 들어보셨을 거라고 생각이 드는데요,"['혁신도시', '에', '대해', '서는', '많이', '들', '들어', '보셨...",혁신도시 에 대해 서는 많이 들 들어 보셨을 거 라고 생각 이 드는데요,안녕하세요 . 메리 성 입니다 혁신도시 에 대해 서는 많이 들 들어 보셨을 거 라고...
4,현재 혁신도시 시즌2가 지속적으로 진행되고 있는 만큼 지금 시기에 이에 대해 이야기...,1,현재 혁신도시 시즌2가 지속해 진행되고 있는 만큼 지금 시기에 이에 대해 이야기해보...,"['현재', '혁신도시', '시즌', '2', '가', '지속', '해', '진행'...",현재 혁신도시 시즌 2 가 지속 해 진행 되고 있는 만큼 지금 시기 에 이 에 대해...,안녕하세요 . 메리 성 입니다 혁신도시 에 대해 서는 많이 들 들어 보셨을 거 라고...
...,...,...,...,...,...,...
164722,의 건립비로 지출하는 기부금 9 중소기업협동조합법 에 따른 중소기업중앙회에 중소기업...,6126,의 건립비로 지출하는 기부금 9 중소기업협동조합법에 따른 중소기업중앙회에 중소기업의...,"['의', '건립', '비', '로', '지출', '하는', '기', '부금', '...",의 건립 비 로 지출 하는 기 부금 9 중소기업 협동조합 법 에 따른 중소기업 중앙...,기획재정부 고시 제 2018 9 호 법인 세법 시행 령 제 36조 제 1 항 제 1...
164723,이하 이 호에서 같다 가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경...,6126,이하 이 호에서 같다가 추천하는 자에게 지출하거나 대한체육회에 운동선수양성 단체경기...,"['이하', '이', '호', '에서', '같다가', '추천', '하는', '자',...",이하 이 호 에서 같다가 추천 하는 자 에게 지출 하 거나 대 한 체육회 에 운동선...,기획재정부 고시 제 2018 9 호 법인 세법 시행 령 제 36조 제 1 항 제 1...
164724,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,6126,21 근로자직업능력 개발법 시행령 제2조에 따른 공공단체에 근로자훈련사업비로 지출하...,"['21', '근로자', '직업', '능력', '개발', '법', '시행', '령'...",21 근로자 직업 능력 개발 법 시행 령 제 2조 에 따른 공공 단체 에 근로자 훈...,기획재정부 고시 제 2018 9 호 법인 세법 시행 령 제 36조 제 1 항 제 1...
164725,한국국제학교 22 천진한국국제학교 23 테헤란한국학교 ㅇ 법정기부금단체 중 전문모금...,6126,한국국제학교 22 천진 한국국제학교 23 테헤란 한국학교 ㅇ 법정기부금 단체 중 전...,"['한국', '국제', '학교', '22', '천진', '한국', '국제', '학교...",한국 국제 학교 22 천진 한국 국제 학교 23 테헤란 한국 학교 ㅇ 법정 기 부금...,기획재정부 고시 제 2018 9 호 법인 세법 시행 령 제 36조 제 1 항 제 1...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,코리아 투데이 뉴스 상주시 상주시 2020년 상반기 정기 인사 이동 조서 2020 ...
1,1,안녕하세요 . 메리 성 입니다 혁신도시 에 대해 서는 많이 들 들어 보셨을 거 라고...
2,2,4 급 승진 경제 기획 국장 박수 원 문화 체육 관광 국장 김 회식 의회 사무국 장...
3,3,안녕하세요 . 반고 유 은기 유진선 입니다 오늘 은 2019년 의 마지막 날 입니다...
4,4,한국 산림 복지 진흥 원 국립 칠곡 숲체 원 사인 간판 디자인 및 시공 https ...
...,...,...
6122,6122,4 급 승진 금오공대 파견 김용수 경제 산업 국장 구 영훈 의회 사무국 장 안민 행...
6123,6123,안녕하세요 . 라포르 커뮤니케이션 원 셈 이에요 2019년 한 해 잘 시작 하셨나요...
6124,6124,블로그 부동산 뉴스 95 개 의 글 부동산 뉴스 목록 열기 2018년 12월 31일...
6125,6125,blog 공지 목록 공지 글 글 제목 작성 일 1 공지 3월 27일 수요일 2019...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(6127, 146837)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)

In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,000100357,000100389,0001561423,000163,00019,0002,...,힘차게,힘차고,힘차네요,힘차다,힘찬,힘찹니다,힙,힙합,힙해,힝
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.012128,0.013952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6122,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6123,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6124,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6125,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 이제 다 완성된 TF-IDF 행렬은 txt로 저장해줌.
tfidf_DF.to_csv("../dataset/본문_TF-IDF/2019산림복지_본문_TF-IDF행렬.txt", index=False, encoding='UTF-8')

In [15]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,47.553048
1,00,68.082215
2,000,20.202526
3,0001,0.083533
4,000100357,0.040675
...,...,...
146832,힘찹니다,0.032328
146833,힙,0.087328
146834,힙합,0.383293
146835,힙해,0.085936


In [16]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/본문_TF-IDF/2019산림복지_본문_TF-IDF합.txt", index=False, encoding='UTF-8')